## Deliverable 2. Create a Customer Travel Destinations Map.

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [34]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,city_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Bathsheba,BB,13.22,-59.52,78.80,88,90,21.92,thunderstorm with light rain
1,1,Carnarvon,AU,-24.87,113.63,71.60,68,0,23.04,clear sky
2,2,Busselton,AU,-33.65,115.33,61.00,78,97,11.01,overcast clouds
3,3,Ushuaia,AR,-54.80,-68.30,28.40,92,90,27.51,heavy snow
4,4,Atuona,PF,-9.80,-139.03,78.49,76,3,20.89,clear sky


In [36]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_input = float(input("Please enter a desired temperature for the trip (F): "))
max_input = float(input("Please enter a desired temperature for the trip (F): "))

Please enter a desired temperature for the trip (F): 75
Please enter a desired temperature for the trip (F): 99


In [37]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
desired_city_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_input) & \
                                   (city_data_df['Max Temp'] <= max_input)]
desired_city_df.head(10)

,city_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Bathsheba,BB,13.22,-59.52,78.80,88,90,21.92,thunderstorm with light rain
4,4,Atuona,PF,-9.80,-139.03,78.49,76,3,20.89,clear sky
24,24,Bubaque,GW,11.28,-15.83,81.86,75,98,6.15,overcast clouds
29,29,Dhidhdhoo,MV,6.88,73.10,81.70,72,22,11.14,few clouds
35,35,Eydhafushi,MV,5.10,73.07,81.95,73,55,12.28,broken clouds
40,40,Georgetown,MY,5.41,100.34,87.80,62,20,4.70,few clouds
41,41,Guerrero Negro,MX,27.98,-114.06,80.02,50,0,9.69,clear sky
47,47,Gigmoto,PH,13.78,124.39,86.23,72,100,4.83,overcast clouds
48,48,Tabuk,PH,17.42,121.44,84.69,73,97,3.42,light rain
50,50,Bengkulu,ID,-3.80,102.27,88.14,59,69,4.07,broken clouds


In [38]:
# 4a. Determine if there are any empty rows.
desired_city_df.count() # or
#desired_city_df.isnull().sum()

city_ID        222
City           222
Country        222
Lat            222
Lng            222
Max Temp       222
Humidity       222
Cloudiness     222
Wind Speed     222
Description    222
dtype: int64

In [39]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no empty rows, assiging clean_df = desired_city_df 

clean_df = desired_city_df.dropna()
clean_df.count()

city_ID        222
City           222
Country        222
Lat            222
Lng            222
Max Temp       222
Humidity       222
Cloudiness     222
Wind Speed     222
Description    222
dtype: int64

In [40]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Bathsheba,BB,78.80,thunderstorm with light rain,13.22,-59.52,
4,Atuona,PF,78.49,clear sky,-9.80,-139.03,
24,Bubaque,GW,81.86,overcast clouds,11.28,-15.83,
29,Dhidhdhoo,MV,81.70,few clouds,6.88,73.10,
35,Eydhafushi,MV,81.95,broken clouds,5.10,73.07,
40,Georgetown,MY,87.80,few clouds,5.41,100.34,
41,Guerrero Negro,MX,80.02,clear sky,27.98,-114.06,
47,Gigmoto,PH,86.23,overcast clouds,13.78,124.39,
48,Tabuk,PH,84.69,light rain,17.42,121.44,
50,Bengkulu,ID,88.14,broken clouds,-3.80,102.27,


In [46]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [49]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Bathsheba,BB,78.80,thunderstorm with light rain,13.22,-59.52,Atlantis Hotel
4,Atuona,PF,78.49,clear sky,-9.80,-139.03,Villa Enata
24,Bubaque,GW,81.86,overcast clouds,11.28,-15.83,Casa Dora - Mango Lodge
29,Dhidhdhoo,MV,81.70,few clouds,6.88,73.10,Dhidhoo beach
35,Eydhafushi,MV,81.95,broken clouds,5.10,73.07,Soneva Fushi
40,Georgetown,MY,87.80,few clouds,5.41,100.34,Cititel Penang
41,Guerrero Negro,MX,80.02,clear sky,27.98,-114.06,"Casa Laguna, Bed & Breakfast"
47,Gigmoto,PH,86.23,overcast clouds,13.78,124.39,Cj's Travellers Inn
48,Tabuk,PH,84.69,light rain,17.42,121.44,Grand Zion Garden Resort Hotel
50,Bengkulu,ID,88.14,broken clouds,-3.80,102.27,Grage Hotel Bengkulu


In [50]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City           222
Country        222
Max Temp       222
Description    222
Lat            222
Lng            222
Hotel Name     222
dtype: int64

In [52]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [63]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather Description with Max Temp</dt><dd>{Description},  {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [64]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))